In [1]:
# !pip3 install malaya -U --no-deps

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import tensorflow as tf
import tensorflow_text  

sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], 'output/1603124082')
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/1603124082/variables/variables


In [4]:
import malaya
malaya.__version__, malaya.__gpu__

('3.9.1', ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'])

In [5]:
model = malaya.true_case.transformer(model = 'small')

In [6]:
from glob import glob
import json

positives = glob('positive/*.json')
positives

['positive/clean-gossipcop_fake.json',
 'positive/clean-politifact_fake.json',
 'positive/positive-fake.json']

In [7]:
texts = []
for p in positives:
    with open(p) as fopen:
        texts.extend(json.load(fopen))
len(texts)

15191

In [8]:
%%time

s = model.true_case([texts[0]])[0]

CPU times: user 6.18 s, sys: 745 ms, total: 6.93 s
Wall time: 2.92 s


In [9]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(s)
string

'Caitlyn Jenner cuba untuk menjaga kehidupan romantiknya secara peribadi. Selepas menghadiri 2018 Espys dengan Sophia Hutchins, peminat telah ingin tahu jika pingat emas telah menemui teman wanita baru. Tetapi apabila bercakap dengan pelbagai untuk isu Hollywood Trans mereka, atlet itu tidak bersedia untuk mendedahkan terlalu banyak maklumat. "Kita tidak akan masuk ke dalamnya," kata Caitlyn kepada penerbitan itu "Tetapi kami sangat dekat, kami melakukan banyak perkara bersama-sama, kami tidak dapat dipisahkan, kami adalah teman terbaik" dalam beberapa minggu kebelakangan ini, persahabatan itu telah dipamerkan menerusi media sosial. Sama ada menikmati padang golf atau mendapat permaidani merah, kedua-dua ini kelihatan menikmati masa bersama. Tetapi pada masa ini, kedua-dua pihak tidak tertumpu pada perhatian media.'

In [10]:
from tqdm import tqdm

r = []
for i in tqdm(range(len(texts))):
    try:
        splitted = texts[i].split()
        if len(splitted) >= 100:
            s = ' '.join(splitted[:800])
            s = model.true_case([s])[0]
            p = pred([f'ringkasan: {cleaning(s)}'] * 5).tolist()
            p = [i.decode() for i in p]
            r.append((s, p))
    except Exception as e:
        print(e)

 35%|███▍      | 5263/15191 [27:26:33<51:46:00, 18.77s/it] 


KeyboardInterrupt: 

In [11]:
with open('fake-news-positives-summarization.json', 'w') as fopen:
    json.dump(r, fopen)